In [348]:
import pandas as pd 
import numpy as np 

import csv
import matplotlib.pyplot as plt
import tqdm
from tqdm.notebook import tqdm_notebook
import itertools

In [29]:
from zipfile import ZipFile 

from os import listdir
from tqdm.notebook import tqdm_notebook
from pathlib import Path 

In [61]:
base = Path("./data")
path2friends = base / "FINAL_FEATURES_FRIENDS.tsv"
path2features = base/ "FINAL_FEATURES_TRAINTEST.tsv"
path2seqmatrix = base / "FINAL_SEQUENCES_MATRIX.tsv"
path2targets = base / "FINAL_TARGETS_DATES_TRAINTEST.tsv"
path2allsequences = base / "FINAL_ALL_SEQUENCES_TRAINTEST.tsv"

## Обработка таблиц с размеченными CLIENT_ID
### И таблиц с сессиями

In [114]:
seq_matrix_df = pd.read_csv('./data/FINAL_SEQUENCES_MATRIX.tsv', 
                         sep=";")
seq_matrix = seq_matrix_df.SESSIONS_SEQUENCES.apply(eval)

In [413]:
targets_dates_traintest_df = pd.read_csv('./data/FINAL_TARGETS_DATES_TRAINTEST.tsv', 
                         sep="\t")
targets_dates_traintest_df['TARGET'].unique()

targets_dates_traintest_df[targets_dates_traintest_df['TARGET'] == 'test'].to_csv(str('./data/clean_data/TEST_TARGETS_DATES.csv'), index=None)
targets_dates_traintest_not_test_df = targets_dates_traintest_df[targets_dates_traintest_df['TARGET'] != 'test']

train_df=targets_dates_traintest_not_test_df.sample(frac=0.8,random_state=42)
val_df=targets_dates_traintest_not_test_df.drop(train_df.index)

train_df.reset_index(drop=True).to_csv(str('./data/clean_data/TRAIN_TARGETS_DATES.csv'), index=None)
val_df.reset_index(drop=True).to_csv(str('./data/clean_data/VAL_TARGETS_DATES.csv'), index=None)

In [93]:
def code_to_dict(seq_matrix):
    dict_int = dict()
    for seq in tqdm_notebook(seq_matrix.to_list()):
        for code in seq:
            if code not in dict_int.keys():
                dict_int[code] = 1
            else:
                dict_int[code] += 1
    return dict_int

In [94]:
seq_matrix_dict_of_code = code_to_dict(seq_matrix=seq_matrix)

  0%|          | 0/1489576 [00:00<?, ?it/s]

In [77]:
print('Процент отношения кол-ва уникальных сессий на сессий всего')
print(len(seq_matrix_dict_of_code)/seq_matrix_dict_of_code.apply(len).sum())

Процент отношения количества сессий на кол-во уникальных
0.012734640825245952


In [105]:
all_seq_matrix_df = pd.read_csv('./data/FINAL_ALL_SEQUENCES_TRAINTEST.tsv',
                         sep="\t")
all_seq_matrix = all_seq_matrix_df.SEQUENCE.apply(eval)

In [87]:
all_seq_matrix_dict_of_code = code_to_dict(seq_matrix=all_seq_matrix)

  0%|          | 0/123707 [00:00<?, ?it/s]

In [102]:
print(f'FINAL_SEQUENCES_MATRIX        {len(set(seq_matrix_dict_of_code.keys()))}')
print(f'FINAL_ALL_SEQUENCES_TRAINTEST {len(set(all_seq_matrix_dict_of_code.keys()))}')
print(f'INTERSECTION                  {len(set(all_seq_matrix_dict_of_code.keys()) & set(seq_matrix_dict_of_code.keys()))}')

FINAL_SEQUENCES_MATRIX        444063
FINAL_ALL_SEQUENCES_TRAINTEST 153528
INTERSECTION                  102741


In [112]:
len(all_seq_matrix_df['CLIENT_ID'].unique()) == len(all_seq_matrix_df)

True

## final features friends
### Таблица с "графом" друзей и фичами связей

In [5]:
with open('./data/FINAL_FEATURES_FRIENDS.tsv', 'r+') as f:

    next(f)
    line = f.readline()
line.split('\t')[0], line.split('\t')[-1]

('999819', '99981973\n')

In [305]:
%%time
N = 1000000
lines = []
clients_friends = dict()
clients_friends_count = dict()
clients_friends_features = dict()

with open('./data/FINAL_FEATURES_FRIENDS.tsv', 'r+') as f:

    next(f)
    for line in tqdm_notebook(f):
        line = list(map(lambda x: float(x), line.split('\t')))
        client_id = int(line[0])
        friend_id = int(line[-1])
        if client_id in clients_friends.keys():
            clients_friends[client_id].append(friend_id)
            clients_friends_features[client_id] += np.array(line[1:-1])
            clients_friends_count[client_id] += 1
        else:
            clients_friends[client_id] = [friend_id]
            clients_friends_features[client_id] = np.array(line[1:-1])
            clients_friends_count[client_id] = 1

0it [00:00, ?it/s]

Wall time: 43min 59s


In [306]:
import pickle
with open('./data/clean_data/clients_friends.pickle', 'wb') as handle:
    pickle.dump(clients_friends, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('./data/clean_data/clients_friends_count.pickle', 'wb') as handle:
    pickle.dump(clients_friends_count, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('./data/clean_data/clients_friends_features.pickle', 'wb') as handle:
    pickle.dump(clients_friends_features, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [316]:
## Кол-во клиентов с данными о друзьях
len(clients_friends_count.keys())

130561

In [331]:
clients_friends_count_df = pd.DataFrame({k: [v] for k,v in clients_friends_count.items()}).T
clients_friends_count_df = clients_friends_count_df.reset_index()
clients_friends_count_df = clients_friends_count_df.rename({0: 'FRIENDS_COUNT', 'index': 'CLIENT_ID'}, axis = 1)
clients_friends_count_df.to_csv('./data/clean_data/clients_friends_count.csv')
clients_friends_count_df.head(2)

,CLIENT_ID,FRIENDS_COUNT
0,999819,80
1,99978,100


In [318]:
clients_friends_features_df = pd.DataFrame(clients_friends_features).T
clients_friends_features_df = clients_friends_features_df.reset_index()
clients_friends_features_df = clients_friends_features_df.rename({'index': 'CLIENT_ID'}, axis = 1)
# clients_friends_features_df.to_csv('./data/clean_data/clients_friends_features.csv')
clients_friends_features_df.head(2)

,CLIENT_ID,0,1,2,3,4,5,6,7,8,...,1004,1005,1006,1007,1008,1009,1010,1011,1012,1013
0,999819,0.0,1310.0,6879.0,52.0,473.0,504.0,2413.0,0.0,60.0,...,125.0,72.0,263.0,0.0,0.0,316.0,0.0,0.0,0.0,0.0
1,99978,0.0,964.0,5932.0,58.0,308.0,309.0,1160.0,0.0,10.0,...,0.0,0.0,250.0,0.0,3.0,1783.0,0.0,0.0,0.0,0.0
2,999555,2.0,209.0,3958.0,112.0,52.0,677.0,847.0,0.0,0.0,...,0.0,8.0,0.0,0.0,62.0,1422.0,0.0,0.0,0.0,0.0
3,999478,16.0,291.0,1448.0,33.0,50.0,255.0,474.0,0.0,0.0,...,0.0,28.0,0.0,0.0,208.0,7637.0,0.0,0.0,0.0,0.0
4,99881,2.0,199.0,1057.0,6.0,22.0,149.0,238.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,453.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130556,1196688,0.0,144.0,1855.0,48.0,22.0,339.0,465.0,0.0,8.0,...,89.0,816.0,0.0,0.0,8525.0,0.0,0.0,0.0,0.0,0.0
130557,1196513,4.0,212.0,1416.0,4.0,38.0,77.0,173.0,0.0,0.0,...,0.0,320.0,0.0,0.0,0.0,29.0,0.0,0.0,0.0,0.0
130558,1196269,0.0,1741.0,1557.0,33.0,32.0,101.0,700.0,0.0,0.0,...,0.0,20.0,0.0,0.0,0.0,6033.0,0.0,0.0,0.0,0.0
130559,1196232,0.0,1102.0,1579.0,3.0,81.0,204.0,372.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,435.0,0.0,0.0,0.0,0.0


In [364]:
## Каждый клиент из датасета с друзьми есть в размеченных данных (но не наоборот)
print('DATASETS LENGTHS')
print(f'''MATRIX        {len(set(targets_dates_traintest_df['CLIENT_ID']))}''')
print(f'''FRIEND        {len(set(clients_friends_count_df['CLIENT_ID']))}''')
print(f'''INSTERSECTION {len(set(targets_dates_traintest_df['CLIENT_ID']) & set(clients_friends_count_df['CLIENT_ID']))}''')

DATASETS LENGTHS
MATRIX        159290
FRIEND        130561
INSTERSECTION 130561


In [363]:
clients_friends_features_df = pd.merge(clients_friends_features_df, clients_friends_count_df, on='CLIENT_ID')
clients_friends_features_df[list(range(0, 1014))] = ( clients_friends_features_df[list(range(0, 1014))].T / clients_friends_features_df['FRIENDS_COUNT'] ).T
clients_friends_features_df.to_csv('./data/clean_data/clients_friends_features.csv')

In [393]:
clients_friends_features_df.head(2)

,CLIENT_ID,0,1,2,3,4,5,6,7,8,...,1005,1006,1007,1008,1009,1010,1011,1012,1013,FRIENDS_COUNT
0,999819,0.0,16.375,85.9875,0.65,5.9125,6.30,30.1625,0.0,0.75,...,0.9,3.2875,0.0,0.00,3.95,0.0,0.0,0.0,0.0,80
1,99978,0.0,9.640,59.3200,0.58,3.0800,3.09,11.6000,0.0,0.10,...,0.0,2.5000,0.0,0.03,17.83,0.0,0.0,0.0,0.0,100


In [391]:
clients_friends_features_df.to_csv('./data/clean_data/clients_friends_features.csv')